In [10]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
import shutil
import torch


In [4]:
input_dir = '/Users/jonaheaton/ReviveMed Dropbox/Jonah Eaton/development_CohortCombination/hilic_pos_2024_feb_05_read_norm_poolmap/subset all_studies with align score 0.25 from Merge_Jan25_align_80_40_fillna_avg/num_cohorts_thresh_0.5/std_1_Multi'
save_dir = '/Users/jonaheaton/Desktop/mskcc_study_feb13'


X_finetune_file = 'X_finetune.csv'
y_finetune_file = 'y_finetune.csv'
X_pretrain_file = 'X_pretrain.csv'

n_repeats = 6
cv_splits = 5
cross_val_seed = 42
y_stratify_col = 'MSKCC'


# save the pretraining data to the save_dir
# X_pretrain = pd.read_csv(os.path.join(input_dir, X_pretrain_file), index_col=0)
# X_pretrain.to_csv(os.path.join(save_dir, 'X_pretrain.csv'))
shutil.copy(os.path.join(input_dir, X_pretrain_file), os.path.join(save_dir, 'X_pretrain.csv'))

X_finetune = pd.read_csv(os.path.join(input_dir, X_finetune_file), index_col=0)
y_finetune = pd.read_csv(os.path.join(input_dir, y_finetune_file), index_col=0)
y_stratify = y_finetune[y_stratify_col]





for iter in range(n_repeats):
    skf = StratifiedKFold(n_splits=cv_splits, random_state=cross_val_seed+iter, shuffle=True)
    skf.get_n_splits(X_finetune, y_stratify)

    # create a subdirectory for each of the cross-validation splits
    for i, (train_index, test_index) in enumerate(skf.split(X_finetune, y_stratify)):

        subset_id = iter * cv_splits + i
        X_train, X_val = X_finetune.iloc[train_index], X_finetune.iloc[test_index]
        y_train, y_val = y_finetune.iloc[train_index], y_finetune.iloc[test_index]
        X_train.to_csv(os.path.join(save_dir, f'X_train_{subset_id}.csv'))
        X_val.to_csv(os.path.join(save_dir, f'X_test_{subset_id}.csv'))
        y_train.to_csv(os.path.join(save_dir, f'y_train_{subset_id}.csv'))
        y_val.to_csv(os.path.join(save_dir, f'y_test_{subset_id}.csv'))


In [5]:
from prep import ClassifierDataset

In [8]:
data_dir = save_dir
finetune_label_col = 'MSKCC'
finetune_label_encoder = {'FAVORABLE': 1, 'POOR': 0, 'INTERMEDIATE': np.nan}
finetune_dataset = ClassifierDataset(data_dir, 
                            subset='train_{}'.format(subset_id),
                            label_col=finetune_label_col,
                            label_encoder=finetune_label_encoder)

In [15]:
y_vals = finetune_dataset.y
# convert tensor to integer

In [16]:
class_weights = 1 / torch.bincount(y_vals.long())

In [17]:
class_weights

tensor([0.0090, 0.0049])

In [9]:
finetune_dataset.y

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1.,
        1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0.,
        1., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1.,
        1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1.,
        1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0.,
        1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
        0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0.,
        0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1.,
        1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0.,
        1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [22]:
train_sz = len(finetune_dataset)
pretrain_batch_size = 23
print(train_sz // pretrain_batch_size)
new_batch_sz = (train_sz // (train_sz // pretrain_batch_size))
print(new_batch_sz)

# what is the remainder?
print(train_sz % pretrain_batch_size)
print(train_sz % new_batch_sz)

13
24
18
5


In [36]:
def get_clean_batch_sz(len_dataset, batch_sz):
    curr_remainder = len_dataset % batch_sz
    new_batch_sz = int(len_dataset/np.ceil(len_dataset / batch_sz))
    new_remainder = len_dataset % new_batch_sz
    print(f'curr_remainder: {curr_remainder}, new_remainder: {new_remainder}')
    return new_batch_sz

In [37]:
get_clean_batch_sz(train_sz, pretrain_batch_size)

curr_remainder: 18, new_remainder: 9


22

In [29]:
train_sz / pretrain_batch_size

13.782608695652174

In [28]:
np.floor(train_sz / pretrain_batch_size)

13.0

In [26]:
train_sz

317

In [25]:
train_sz

317